# Hasztagiem na początku kodu jest opisane który numer kolumny zmieniam żeby jakoś to ten

In [ ]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

In [3]:
df = pd.read_csv('train.csv')

### Tutaj sobie zdefiniowałem taką funkcję żeby kilka razy tego nie pisać, to usuwa po prostu "_"

In [4]:
def Object_to_Numeric(df, s):
    return pd.to_numeric(df[s].str.replace('_', ''))

### Tutaj kilka łatwych zamian

In [ ]:
#19, 24, 16, 26
df['Outstanding_Debt'] = Object_to_Numeric(df, 'Outstanding_Debt')
df['Amount_invested_monthly'] = Object_to_Numeric(df, 'Amount_invested_monthly')
df['Changed_Credit_Limit'] = Object_to_Numeric(df, 'Changed_Credit_Limit')
df['Monthly_Balance'] = df['Monthly_Balance'].str.replace('_', '')
    df = df.astype({'Monthly_Balance': 'float'}) # Monthly_Balance jakoś nie chciał współpracować,
                                                 # ale jest git

### Tu jakieś szalone rzeczy, ale tylko dla kolumny 'Credit_History_Age'

In [5]:
#21
def History_Age_2_months(df):
    df['Credit_History_Age'] = df['Credit_History_Age'].str.replace(' Months', '')
    return df['Credit_History_Age'].str.replace('\D+', '* 12 +')
df['Credit_History_Age'] = History_Age_2_months(df)
df['Credit_History_Age'].loc[df.Credit_History_Age.isnull()] = df['Credit_History_Age'].loc[df.Credit_History_Age.isnull()].apply(
        lambda x: '-1') #Brak danych jest teraz -1
df['Credit_History_Age'] = df['Credit_History_Age'].apply(eval)

C:\Users\wojte\AppData\Local\Temp\ipykernel_22064\1080241939.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  return df['Credit_History_Age'].str.replace('\D+', '* 12 +')
c:\users\wojte\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


## One-Hot dla Paymentów

In [ ]:
#22, 25
one_hot = pd.get_dummies(df['Payment_of_Min_Amount'])
    df = df.drop('Payment_of_Min_Amount', axis=1)
    df = df.join(one_hot)

    one_hot = pd.get_dummies(df['Payment_Behaviour'])
    df = df.drop('Payment_Behaviour', axis=1)
    df = df.join(one_hot)

#### Ostre dymy z typami kredytów

In [ ]:
#13
df['Type_of_Loan'] = df['Type_of_Loan'].str.replace('and ', '')
    df.Type_of_Loan = df.Type_of_Loan.str.split(', ')
    df['Type_of_Loan'].loc[df.Type_of_Loan.isnull()] = df['Type_of_Loan'].loc[df.Type_of_Loan.isnull()].apply(lambda x: [])
    mlb = MultiLabelBinarizer(sparse_output=True)
    df = df.join(
        pd.DataFrame.sparse.from_spmatrix(
            mlb.fit_transform(df.pop('Type_of_Loan')),
            index=df.index,
            columns=mlb.classes_))
    df.rename(columns={'!@9#%8': 'No_Payment_Data',
                       'No': 'Min_Amount_Not_Paid',
                       'Yes': 'Min_Amount_Paid',
                       }, inplace=True)